In [1]:
import numpy as np

In [2]:
def nms(detections: np.ndarray, confidence_threshold: float=0.25) -> list:
    """后处理

    Args:
        detections (np.ndarray): 检测到的数据 [25200, 85]
        confidence_threshold (float): prediction[4] 是否有物体得分阈值
    """
    boxes = []  # [[xmin, ymin, w, h]]
    class_ids = []
    confidences = []
    for prediction in detections:
        confidence = prediction[4].item()           # 是否有物体得分
        if confidence >= confidence_threshold:      # 是否有物体预支
            classes_scores = prediction[5:]         # 取出所有类别id
            class_id = np.argmax(classes_scores)    # 找到概率最大的id
            if (classes_scores[class_id] > .25):    # 最大概率必须大于 0.25
                confidences.append(confidence)      # 保存置信度(注意保存的是confidence，不是classes_scores[class_id]),类别id,box
                class_ids.append(class_id)
                # center_x, center_y, w, h
                x, y, w, h = prediction[0].item(), prediction[1].item(), prediction[2].item(), prediction[3].item()
                xmin = x - (w / 2)
                ymin = y - (h / 2)
                box = [xmin, ymin, w, h]
                boxes.append(box)

In [7]:
def nms1(detections: np.ndarray, confidence_threshold: float=0.25) -> list:
    """后处理

    Args:
        detections (np.ndarray): 检测到的数据 [25200, 85]
        confidence_threshold (float): prediction[4] 是否有物体得分阈值
        score_threshold (float):      nms分类得分阈值
        nms_threshold (float):        非极大值抑制iou阈值
    """
    # 通过置信度过滤一部分框
    detections = detections[detections[:, 4] > confidence_threshold]
    # 置信度
    confidence = detections[:, 4]
    # 位置坐标
    loc = detections[:, :4]
    # 分类
    cls = detections[:, 5:]
    # 最大分类index
    max_index = cls.argmax(axis=-1)
    # 最大分类score
    max_cls_score = cls.max(axis=-1)
    # 置信度    (n,)
    confidences = confidence[max_cls_score > .25]
    # 类别index (n,)
    class_ids = max_index[max_cls_score > .25]
    # 位置      (n, 4)
    boxes = loc[max_cls_score > .25]
    # center_x, center_y, w, h -> x_min, y_min, w, h
    boxes[:, 0] -= boxes[:, 2] / 2
    boxes[:, 1] -= boxes[:, 3] / 2

In [8]:
x = np.random.normal(0, 1, (25200, 85))
x.shape

(25200, 85)

In [9]:
%timeit nms(x)

60.8 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit nms1(x)

7.51 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
detections = x

In [72]:
detections = detections[detections[:, 4] > 0.25]
detections.shape

(10050, 85)

In [73]:
confidence = detections[:, 4]
confidence.shape

(10050,)

In [74]:
loc = detections[:, :4]
loc.shape

(10050, 4)

In [75]:
cls = detections[:, 5:]
cls.shape

(10050, 80)

In [83]:
max_index = cls.argmax(axis=-1)
max_index.shape, max_index[:10]

((10050,), array([54, 35, 57, 58, 16, 52, 65, 26, 21, 49], dtype=int64))

In [87]:
cls[:, 0].shape

(10050,)

In [77]:
max_cls_score = cls.max(axis=-1)
max_cls_score.shape

(10050,)

In [78]:
# 置信度
confidence[max_cls_score > 3].shape

(1069,)

In [62]:
# 类别index
max_index[max_cls_score > 3].shape

(986,)

In [63]:
loc[max_cls_score > 3].shape

(986, 4)